In [1]:
import re
import opencc
import pandas as pd

from bs4 import BeautifulSoup

df = pd.read_csv("./corpus/_正體(繁體)/中華成語大詞典.csv")
df

,key,value
0,一丁不識,<br>【拼音】： \r\n<br>【解釋】：形容一個字也不認識。 \r\n<br>【...
1,一丁至微,<br>【拼音】： \r\n<br>【解釋】：一丁：一個釘子；至微：極其微小。一個極其...
2,一丁點兒,<br>【拼音】： \r\n<br>【解釋】：形容極少或極小。 \r\n<br>【出處...
3,一不作，二不休,<br>【拼音】： ， \r\n<br>【解釋】：要麼不做，做了就索興做到底。指事情既...
4,一不做，二不休,<br>【拼音】： ， \r\n<br>【解釋】：原意是要麼不做，做了就索興做到底。指...
...,...,...
49445,龜鶴遐齡,<br>【拼音】： \r\n<br>【解釋】：遐：長久。遐齡：高壽。傳說龜、鶴都能活一...
49446,龜齡鶴算,<br>【拼音】： \r\n<br>【解釋】：比喻人之長壽。或用作祝壽之詞。同“龜年鶴...
49447,龜龍片甲,<br>【拼音】： \r\n<br>【解釋】：比喻無論巨細都搜羅進來的好東西。 \r\...
49448,龜龍鱗鳳,<br>【拼音】： \r\n<br>【解釋】：傳統上用來象征高壽、尊貴、吉祥的四種動物...


In [2]:
print(df.loc[0, "value"])

<br>【拼音】：    
<br>【解釋】：形容一個字也不認識。 
<br>【出處】：《舊唐書·張弘靖傳》：“天下無事，汝輩挽得兩石弓，不如識一丁字。” 
<br>【示例】：而雲古無類書，此真～之無知妄作也矣。 ★清·平步青《霞外捃屑》卷七 
<br>【近義詞】：目不識丁 
<br>【反義詞】：學富五車 
<br>【語法】：作謂語、定語；指不識字 
<br>【英文】：not know one's ABC



In [3]:
def clean_all_html_tags(raw_text: str):
    soup = BeautifulSoup(raw_text)
    p_text = soup.get_text().strip()
    return p_text

def sub_tilde_to_word(keyword, text):
    return re.sub('～|\~', keyword, text)

def clean_CRLF(text):
    return text.rstrip() # remove \r\n

In [4]:
tasks_instructions = {
    '解釋': "解釋這個詞",
    '出處': "{}的出處",
    '示例': "造一個句子",
    '近義詞': "舉出一個近似詞",
    '反義詞': "舉出一個反義詞",
    '歇後語': "造一句幽趣有趣的歇後語",
    '語法': "什麼時候會用到{}這個詞",
    '英文': "翻譯成中文",
    '日文': "翻譯成中文",
    '德文': "翻譯成中文",
    '法文': "翻譯成中文",
    '俄文': "翻譯成中文",
    '成語故事': "{}的典故",
}

task_category_mappings = {
    '解釋': 'interpret',
    '出處': 'provenance',
    '示例': 'make_sentence',
    '近義詞': 'synonym',
    '反義詞': 'antonym',
    '歇後語': 'xiehouyu',
    '語法': 'syntax',
    # '簡轉繁': 'translate_from_SC',
    '英文': "translate_from_others",
    '日文': "translate_from_others",
    '德文': "translate_from_others",
    '法文': "translate_from_others",
    '俄文': "translate_from_others",
    '成語故事': 'allusion'
}

In [6]:
category_lst = []
instruction_lst = []
input_lst = []
output_lst = []

instruction_dataset = pd.DataFrame()

for index, row in df.iterrows():
    key = row["key"]
    value = row["value"]

    entries = re.findall("【(.*)】：(.*)", sub_tilde_to_word(key, clean_all_html_tags(value)))
    entries = dict((x, clean_CRLF(y)) for x, y in entries)

    for k, v in entries.items():
        if k == '拼音' or not v: # remove 拼音 & null text
            continue
        else:
            category_lst.append(task_category_mappings[k])

            if "{}" in tasks_instructions[k]:
                instruction_lst.append(tasks_instructions[k].format(key))
                input_lst.append("")
                output_lst.append(entries[k])
            else:
                if '文' in k: # translate task
                    instruction_lst.append(tasks_instructions[k])
                    input_lst.append(entries[k])
                    output_lst.append(key)  
                else:
                    instruction_lst.append(tasks_instructions[k])
                    input_lst.append(key)
                    output_lst.append(entries[k])

instruction_dataset['category'] = category_lst
instruction_dataset['instruction'] = instruction_lst
instruction_dataset['input'] = input_lst
instruction_dataset['output'] = output_lst

In [7]:
instruction_dataset

,category,instruction,input,output
0,interpret,解釋這個詞,一丁不識,形容一個字也不認識。
1,provenance,一丁不識的出處,,《舊唐書·張弘靖傳》：“天下無事，汝輩挽得兩石弓，不如識一丁字。”
2,make_sentence,造一個句子,一丁不識,而雲古無類書，此真一丁不識之無知妄作也矣。 ★清·平步青《霞外捃屑》卷七
3,synonym,舉出一個近似詞,一丁不識,目不識丁
4,antonym,舉出一個反義詞,一丁不識,學富五車
...,...,...,...,...
230521,syntax,什麼時候會用到龜龍鱗鳳這個詞,,作主語、賓語、定語；用於比喻句
230522,interpret,解釋這個詞,龜龍麟鳳,傳統上用來象征高壽、尊貴、吉祥的四種動物。比喻身處高位德蓋四海的人。
230523,provenance,龜龍麟鳳的出處,,東漢·班固《漢書·翟方進傳》：“太皇太後臨政，有龜龍麟鳳之應。”
230524,make_sentence,造一個句子,龜龍麟鳳,風雨以時，草木不落，龜龍麟鳳在效藪矣。 ★唐·陳子昂《諫政理書》


In [8]:
instruction_dataset[instruction_dataset['instruction'].str.contains('翻譯')]

,category,instruction,input,output
6,translate_from_others,翻譯成中文,not know one's ABC,一丁不識
15,translate_from_others,翻譯成中文,a wee bit,一丁點兒
21,translate_from_others,翻譯成中文,all-or-nothing,一不作，二不休
29,translate_from_others,翻譯成中文,as well be hanged/hung for a sheep as/for a lamb,一不做，二不休
30,translate_from_others,翻譯成中文,毒(どく)を食(く)らわばさらま,一不做，二不休
...,...,...,...,...
230393,translate_from_others,翻譯成中文,walk like a dragon and pace like a tiger,龍驤虎步
230398,translate_from_others,翻譯成中文,prance like the dragon and glance like the tig...,龍驤虎視
230436,translate_from_others,翻譯成中文,buster,龐然大物
230437,translate_from_others,翻譯成中文,極(きわ)めて巨大(きょだい)なもの，巨大なしろもの,龐然大物


In [9]:
instruction_dataset[instruction_dataset['input'].str.contains('童顏')]

,category,instruction,input,output
158321,interpret,解釋這個詞,童顏鶴發,顏：臉色。仙鶴羽毛似雪白的頭發，孩子似的紅潤的面色。形容老年人氣色好。
158323,make_sentence,造一個句子,童顏鶴發,（華佗）童顏鶴發，飄然有出世之姿。 ★明·羅貫中《三國演義》第十五回
158324,synonym,舉出一個近似詞,童顏鶴發,鶴發童顏
227117,interpret,解釋這個詞,鶴發童顏,仙鶴羽毛般雪白的頭發，兒童般紅潤的面色。形容老年人氣色好。
227119,make_sentence,造一個句子,鶴發童顏,（華佗）童顏鶴發，飄然有出世之姿。 ★明·羅貫中《三國演義》第十五回
227120,synonym,舉出一個近似詞,鶴發童顏,返老還童、老當益壯
227121,antonym,舉出一個反義詞,鶴發童顏,未老先衰、老態龍鍾


In [10]:
instruction_dataset[instruction_dataset['instruction'].str.contains('歇後語') & instruction_dataset['input'].str.contains('發')]

,category,instruction,input,output
4663,xiehouyu,造一句幽趣有趣的歇後語,一觸即發,上了弦的箭；眼前埋地雷
4691,xiehouyu,造一句幽趣有趣的歇後語,一言不發,鐵水封口
23984,xiehouyu,造一句幽趣有趣的歇後語,借題發揮,指著禿子罵和尚
37721,xiehouyu,造一句幽趣有趣的歇後語,千鈞一發,一根頭發系石磨
54453,xiehouyu,造一句幽趣有趣的歇後語,大發雷霆,暴雨前的閃電
145676,xiehouyu,造一句幽趣有趣的歇後語,百發百中,黃忠射箭；瘌痢頭上打蒼蠅
160647,xiehouyu,造一句幽趣有趣的歇後語,紅得發紫,火盆裏的木炭
173250,xiehouyu,造一句幽趣有趣的歇後語,舊病複發,關節炎遇上連陰天
217317,xiehouyu,造一句幽趣有趣的歇後語,頭皮發麻,花椒水洗腦袋


In [11]:
# 可以發現此語料品質不佳，許多是機翻，因此剔除掉非對稱簡繁字雖然粗暴，但保證了正體語料的 100% 正確性
not_symmetric_characters_in_SC_to_TC_df = pd.read_csv('./corpus/非對稱簡繁字/簡化字對照標準字總表.csv')

not_symmetric_characters = []
not_symmetric = not_symmetric_characters_in_SC_to_TC_df[not_symmetric_characters_in_SC_to_TC_df['非對稱簡繁字']]

for s in not_symmetric['正體字'].tolist():
    l = eval(s)  # NOTE: the input is a fixed file, so I use `eval`
    not_symmetric_characters.extend(l)

In [12]:
category_lst = []
instruction_lst = []
input_lst = []
output_lst = []

not_contains_symmetric_characters_dataset = pd.DataFrame()

not_symmetric_characters_regex = "[" + '|'.join(not_symmetric_characters) + "]"

for index, row in instruction_dataset.iterrows():
    _category = row['category']
    _instruction = row['instruction']
    _input = row['input']
    _output = row['output']

    if re.search(not_symmetric_characters_regex, _input) or re.search(not_symmetric_characters_regex, _output):
        continue
    else:
        category_lst.append(_category)
        instruction_lst.append(_instruction)
        input_lst.append(_input)
        output_lst.append(_output)

not_contains_symmetric_characters_dataset['category'] = category_lst
not_contains_symmetric_characters_dataset['instruction'] = instruction_lst
not_contains_symmetric_characters_dataset['input'] = input_lst
not_contains_symmetric_characters_dataset['output'] = output_lst

In [13]:
not_contains_symmetric_characters_dataset

,category,instruction,input,output
0,interpret,解釋這個詞,一丁不識,形容一個字也不認識。
1,synonym,舉出一個近似詞,一丁不識,目不識丁
2,antonym,舉出一個反義詞,一丁不識,學富五車
3,syntax,什麼時候會用到一丁不識這個詞,,作謂語、定語；指不識字
4,translate_from_others,翻譯成中文,not know one's ABC,一丁不識
...,...,...,...,...
111071,syntax,什麼時候會用到龜齡鶴算這個詞,,作賓語、定語；用於祝壽等
111072,provenance,龜龍片甲的出處,,唐·張懷瓘《書議》：“麟鳳一毛，龜龍片甲，亦無所不錄。”
111073,syntax,什麼時候會用到龜龍鱗鳳這個詞,,作主語、賓語、定語；用於比喻句
111074,make_sentence,造一個句子,龜龍麟鳳,風雨以時，草木不落，龜龍麟鳳在效藪矣。 ★唐·陳子昂《諫政理書》


In [14]:
t2s_converter = opencc.OpenCC('t2s.json')

"""traditional chinese to simplified chinese"""
def convert_TC_to_SC(zhtw_text: str, converter=None):
    if not converter:
        converter = opencc.OpenCC('t2s.json')
    return converter.convert(zhtw_text)

In [15]:
TC_corpus = list(set(not_contains_symmetric_characters_dataset['output']))
SC_corpus = list(map(lambda x: convert_TC_to_SC(x, t2s_converter), TC_corpus))
categories = ["translate_from_SC"] * len(TC_corpus) # '簡轉繁': 'translate_from_SC'
instructions = ["翻譯成中文"] * len(TC_corpus)

In [16]:
translate_SC_to_TC_df = pd.DataFrame(
    data=list(zip(categories, instructions, SC_corpus, TC_corpus)),
    columns=['category', 'instruction', 'input', 'output']
)

In [17]:
chinese_idiom_dictionary_df = pd.concat([not_contains_symmetric_characters_dataset, translate_SC_to_TC_df])
chinese_idiom_dictionary_df

,category,instruction,input,output
0,interpret,解釋這個詞,一丁不識,形容一個字也不認識。
1,synonym,舉出一個近似詞,一丁不識,目不識丁
2,antonym,舉出一個反義詞,一丁不識,學富五車
3,syntax,什麼時候會用到一丁不識這個詞,,作謂語、定語；指不識字
4,translate_from_others,翻譯成中文,not know one's ABC,一丁不識
...,...,...,...,...
75967,translate_from_SC,翻譯成中文,绿油油的草好像地上铺的褥子。常指可供临时休憩的草地。,綠油油的草好像地上鋪的褥子。常指可供臨時休憩的草地。
75968,translate_from_SC,翻譯成中文,他山攻玉,他山攻玉
75969,translate_from_SC,翻譯成中文,作宾语；比喻战争的前线,作賓語；比喻戰爭的前線
75970,translate_from_SC,翻譯成中文,作宾语、定语；指纷乱的样子,作賓語、定語；指紛亂的樣子


In [18]:
chinese_idiom_dictionary_df.to_csv("./instruction_datasets/chinese_idiom_dictionary.csv", index=0)